<a href="https://colab.research.google.com/github/PeterLee2314/Fruit-Spoilage-Detection-Application/blob/main/fruit_spoilage_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Model will be trained by Google Colab in this juypter notebook

In [ ]:
import sys
import tensorflow as tf

print(sys.version)
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
setup = "clone"
#Environment Setup (clone)

!git clone https://github.com/PeterLee2314/Fruit-Spoilage-Detection-Application.git
!git clone https://github.com/PeterLee2314/Fruit-Spoilage-Detection-Dataset.git


#Environment setup (cloud)
from google.colab import drive
drive.mount('/content/drive/')


In [ ]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split

# Define the source and destination directories
'''
source_dir = '/content/Fruit-Spoilage-Detection-Application/dataset/complete'
train_dir = '/content/Fruit-Spoilage-Detection-Application/dataset/train'
test_dir = '/content/Fruit-Spoilage-Detection-Application/dataset/test'
valid_dir = '/content/Fruit-Spoilage-Detection-Application/dataset/validation'
'''
source_dir = '/content/Fruit-Spoilage-Detection-Dataset/dataset/complete'
train_dir = '/content/Fruit-Spoilage-Detection-Dataset/dataset/train'
test_dir = '/content/Fruit-Spoilage-Detection-Dataset/dataset/test'
valid_dir = '/content/Fruit-Spoilage-Detection-Dataset/dataset/validation'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)


# source_directories = ['/content/Fruit-Spoilage-Detection-Application/dataset/complete']
source_directories = ['/content/Fruit-Spoilage-Detection-Dataset/dataset/complete']


#source_directories = ['/content/Fruit-Spoilage-Detection-Application/dataset/complete', '/content/drive/MyDrive/TensorFlow FYP/complete']
# Complete -> test/train/validation -> tmp -> test/train/validation (move[prevent repeated image])
subdirectories = ['bad_apple', 'bad_banana', 'bad_orange',
                  'good_apple', 'good_banana', 'good_orange']

#subdirectories = ['bad_apple', 'good_apple']

# Split ratios for train, test, and validation datasets (70% train, 20% test, 10% validation)
train_size = 0.7
test_size = 0.1
valid_size = 0.2

# Forced seeding
random.seed(42)
for source_dir in source_directories:
  for subdirectory in subdirectories:
      subdirectory_path = os.path.join(source_dir, subdirectory)
      sub_subdirectories = [name for name in os.listdir(subdirectory_path) if os.path.isdir(os.path.join(subdirectory_path, name))]

      train_subdirectory_dir = os.path.join(train_dir, subdirectory)
      test_subdirectory_dir = os.path.join(test_dir, subdirectory)
      valid_subdirectory_dir = os.path.join(valid_dir, subdirectory)

      os.makedirs(train_subdirectory_dir, exist_ok=True)
      os.makedirs(test_subdirectory_dir, exist_ok=True)
      os.makedirs(valid_subdirectory_dir, exist_ok=True)

      # (Optional Move/Copy) pattern + "/*/*" with glob.glob(pattern) for copy
      # for sub_subdirectory in sub_subdirectories:
      #   pattern = os.path.join(subdirectory_path,sub_subdirectory) + "/*/*"
      #   for img in glob.glob(pattern):
      #       shutil.copy(img, destination_path)

      for sub_subdirectory in sub_subdirectories:
          sub_subdirectory_path = os.path.join(subdirectory_path, sub_subdirectory)
          files = os.listdir(sub_subdirectory_path)
          print("sub_subdirectory_path",sub_subdirectory_path)
          # Split the files into train, test, and validation sets
          train_files, test_valid_files = train_test_split(files, test_size=(test_size + valid_size), random_state=42, shuffle=True)
          test_files, valid_files = train_test_split(test_valid_files, test_size=(valid_size / (test_size + valid_size)), random_state=42, shuffle=True)

          # Move all images inside train/test/validation folder
          for file in train_files:
              src = os.path.join(sub_subdirectory_path, file)
              dst = os.path.join(train_subdirectory_dir, file)
              shutil.move(src, dst)
              #shutil.copy(src, dst)

          for file in test_files:
              src = os.path.join(sub_subdirectory_path, file)
              dst = os.path.join(test_subdirectory_dir, file)
              shutil.move(src, dst)
              #shutil.copy(src, dst)

          for file in valid_files:
              src = os.path.join(sub_subdirectory_path, file)
              dst = os.path.join(valid_subdirectory_dir, file)
              shutil.move(src, dst)
              #shutil.copy(src, dst)
      #Print len for each subdirectories in train/test/validation folder
      train_count = len(os.listdir(train_subdirectory_dir))
      test_count = len(os.listdir(test_subdirectory_dir))
      valid_count = len(os.listdir(valid_subdirectory_dir))

      print(f"Number of files in {train_subdirectory_dir}: {train_count}")
      print(f"Number of files in {test_subdirectory_dir}: {test_count}")
      print(f"Number of files in {valid_subdirectory_dir}: {valid_count}")

In [ ]:
import os
import shutil

# Define the source and destination directories
temp_directory = '/content/tmp'

# Create the temp directory if it doesn't exist
os.makedirs(temp_directory, exist_ok=True)

# Move data from train_dir to temp_directory
train_fresh_dir = os.path.join(temp_directory, 'train', 'fresh')
train_spoiled_dir = os.path.join(temp_directory, 'train', 'spoiled')
os.makedirs(train_fresh_dir, exist_ok=True)
os.makedirs(train_spoiled_dir, exist_ok=True)

for subdirectory in os.listdir(train_dir):
    train_subdirectory_dir = os.path.join(train_dir, subdirectory)
    for file in os.listdir(train_subdirectory_dir):
        src = os.path.join(train_subdirectory_dir, file)
        dst = os.path.join(train_fresh_dir if 'good' in subdirectory else train_spoiled_dir, file)
        shutil.move(src, dst)

# Move data from test_dir to temp_directory
test_fresh_dir = os.path.join(temp_directory, 'test', 'fresh')
test_spoiled_dir = os.path.join(temp_directory, 'test', 'spoiled')
os.makedirs(test_fresh_dir, exist_ok=True)
os.makedirs(test_spoiled_dir, exist_ok=True)

for subdirectory in os.listdir(test_dir):
    test_subdirectory_dir = os.path.join(test_dir, subdirectory)
    for file in os.listdir(test_subdirectory_dir):
        src = os.path.join(test_subdirectory_dir, file)
        dst = os.path.join(test_fresh_dir if 'good' in subdirectory else test_spoiled_dir, file)
        shutil.move(src, dst)

# Move data from valid_dir to temp_directory
valid_fresh_dir = os.path.join(temp_directory, 'validation', 'fresh')
valid_spoiled_dir = os.path.join(temp_directory, 'validation', 'spoiled')
os.makedirs(valid_fresh_dir, exist_ok=True)
os.makedirs(valid_spoiled_dir, exist_ok=True)

for subdirectory in os.listdir(valid_dir):
    valid_subdirectory_dir = os.path.join(valid_dir, subdirectory)
    for file in os.listdir(valid_subdirectory_dir):
        src = os.path.join(valid_subdirectory_dir, file)
        dst = os.path.join(valid_fresh_dir if 'good' in subdirectory else valid_spoiled_dir, file)
        shutil.move(src, dst)